In [1]:
import selenium 
from selenium import webdriver
import csv # to save the data in csv
import time
from datetime import datetime #to get the current date
import requests
from bs4 import BeautifulSoup 
import pandas as pd

def get_url(position,location):
    position = position.replace(" ","%20")
    location = location.replace(" ","%20")
    template = 'https://www.monster.com.my/srp/results?query={}&locations={}'
    url= template.format(position,location)
    return url

# create function to fetch all info
def get_jobs(card):
    
    job_title = card.div.a.text.strip()
    
    anchor_tag = card.h3.a
    job_url = anchor_tag.get("href")
    job_url = "https:"+ anchor_tag.get('href')
    
    company_name = card.find('span','company-name').text.strip()
    
    job_location = card.find('span','loc').text.strip()
    
    try:
        summary = card.find("p","job-descrip").text.strip()
        test_name = summary.encode('ascii','ignore')
        job_summary = str(test_name,'utf-8').strip()
    except AttributeError:
        job_summary = ''
        
    try:
        job_salary = card.find("span","loc salarySnb").text.strip()
    except AttributeError:
        job_salary = ''

    date_post = card.find("span","posted").text.strip()
    post_date = date_post.split("Posted: ")[1]

    
    today = datetime.today().strftime('%Y-%m-%d')
    
    job_info = (job_title,job_url,company_name,job_location,job_summary,job_salary,post_date ,today)
    
    return job_info

def main(position,location):
    records = []
    url = get_url(position,location)
    
    driver = webdriver.Chrome(executable_path='D:\driver\chromedriver_win32\chromedriver.exe ')
    url = 'https://www.monster.com.my/srp/results?query=Java%20Developer' # driver will open the browser
    driver.get(url)
#      ## pausing the python ,let the website to be open first
    time.sleep(5)
    driver.maximize_window()
    # define an initial value
    temp_height = 0
    
    while True:
        #Looping down the scroll bar
        driver.execute_script("window.scrollBy(0,1000)")
        #sleep and let the scroll bar react
        time.sleep(5)
        #Get the distance of the current scroll bar from the top
        check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
        #If the two are equal to the end
        if check_height==temp_height:
            break
        temp_height=check_height
        
    time.sleep(10)   
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    
    product_cards = soup.find_all('div','card-panel apply-panel job-apply-card')
    
    for everyCard in product_cards:
        jobDetails = get_jobs(everyCard)
        records.append(jobDetails)
        
    cols = ["Job_Name","Job_URL", "Company_Name", "Job_Location",
           "Job_Summary","Job_salary","Post_Date", "Today_Date"]
     
    monster_data = pd.DataFrame(records, columns = cols)
    monster_data.to_csv("D:\\monster1.csv", encoding ="utf-8")
    

In [3]:
main("Java Developer","Kuala Lumpur")
